In [ ]:
'''**The script compiles a list, functions_to_execute, which aggregates several predefined functions updating a financial database.
This list is designed to systematically execute a series of operations aimed at maintaining the currency and integrity of the database's data.
It initiates by fetching symbols and their last update dates from a specific table, a crucial step for determining the scope of new data to be fetched. 
However, due to parameter requirements for certain functions within the list, 
such as fetching new stock data for a given symbol and inserting this new data into the database, 
the script intelligently differentiates between functions that can be executed directly and those that necessitate additional input.
For functions without parameter requirements, the script executes them sequentially, ensuring a streamlined update process.
This methodical approach allows for the efficient management of financial data, ensuring the database reflects the most current information available,
while also acknowledging the need for flexibility in executing functions with specific data requirements.**'''







In [29]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import psycopg2
import configparser
import nbimporter
from General import *
from Functions import *
from SQL import *
from Daily_etl import *
from Connection_to_pgAdmin4 import *

In [30]:
#This code will run the function in one go. 
#The function contain all the functions that will run to update the Warehouse.
#This is part of the automation Objective

In [1]:
def execute_update_process():
    symbols_dates_df = fetch_symbols_and_last_dates()
    if symbols_dates_df.empty:
        print("No symbols or last dates found.")
        return

    all_new_stock_data = pd.DataFrame()
    for index, row in symbols_dates_df.iterrows():
        symbol = row['symbol']
        last_date = row['lastdate']
        start_date = (pd.to_datetime(last_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        symbol_new_data = fetch_new_stock_data_for_symbol(symbol, start_date)

        if symbol_new_data.empty:
            print(f"No new data found for {symbol} since {start_date}. Skipping...")
            continue

        all_new_stock_data = pd.concat([all_new_stock_data, symbol_new_data], ignore_index=True)

    if not all_new_stock_data.empty:
        insert_new_stock_data(all_new_stock_data)
    else:
        print("No new stock data was found for any symbols.")

    insert_into_financial_info()
    update_facts_table()
    update_dates_of_stock_info()
    print("ETL update process completed.")




'''
def execute_update_process():
    # Fetch symbols and their last update dates.
    symbols_dates_df = fetch_symbols_and_last_dates()
    if symbols_dates_df.empty:
        print("No symbols or last dates found.")
        return
    
    # Check if the DataFrame is not empty to proceed.
    if not symbols_dates_df.empty:
        # Initialize an empty DataFrame to store new stock data for all symbols.
    
    all_new_stock_data = pd.DataFrame()

    # Iterate over each row in the DataFrame to get the symbol and its last update date.
    for index, row in symbols_dates_df.iterrows():
        symbol = row['symbol']
        last_date = row['lastdate']
        
        # Calculate the start date as the day after the last recorded date.
        start_date = (pd.to_datetime(last_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')

        # Fetch new stock data for the symbol starting from the calculated start_date.
        symbol_new_data = fetch_new_stock_data_for_symbol(symbol, start_date)

        # Only aggregate this new stock data into a single DataFrame if it's not empty.
        if not symbol_new_data.empty:
            all_new_stock_data = pd.concat([all_new_stock_data, symbol_new_data], ignore_index=True)
        else:
            print(f"No new data found for {symbol} since {start_date}. Skipping...")
            
        # Insert the aggregated new stock data into the database if not empty.
        if not all_new_stock_data.empty:
            insert_new_stock_data(all_new_stock_data)
            print('Symbols extracted from DB. New records extracted. Inserted records to DB.')

    # Execute other updates that don't depend on the fetched new stock data.
    insert_into_financial_info()
    print('Inserted unique rows to the dimension.d_finance_info from historical_stock_data.')
    update_facts_table()
    print('Truncated Last facts table, rebuilt f_usage_daily from dimensions.')
    update_dates_of_stock_info()
    print('Truncated facts.dates_of_stock_info, rebuilt it using f_usage_daily.')
    
    current_date = datetime.now().date()
    print(f"ETL update Complete {current_date}")


'''

'\ndef execute_update_process():\n    # Fetch symbols and their last update dates.\n    symbols_dates_df = fetch_symbols_and_last_dates()\n    if symbols_dates_df.empty:\n        print("No symbols or last dates found.")\n        return\n    \n    # Check if the DataFrame is not empty to proceed.\n    if not symbols_dates_df.empty:\n        # Initialize an empty DataFrame to store new stock data for all symbols.\n    \n    all_new_stock_data = pd.DataFrame()\n\n    # Iterate over each row in the DataFrame to get the symbol and its last update date.\n    for index, row in symbols_dates_df.iterrows():\n        symbol = row[\'symbol\']\n        last_date = row[\'lastdate\']\n        \n        # Calculate the start date as the day after the last recorded date.\n        start_date = (pd.to_datetime(last_date) + pd.Timedelta(days=1)).strftime(\'%Y-%m-%d\')\n\n        # Fetch new stock data for the symbol starting from the calculated start_date.\n        symbol_new_data = fetch_new_stock_data_

In [38]:
execute_update_process()

Daily_etl.ipynb:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  "import configparser\n",


      symbol    lastdate
0     XUEK.L  2024-02-28
1     0NOL.L  2024-02-29
2     0KVR.L  2024-02-29
3     ASIT.L  2024-02-29
4     N30X.L  2014-08-01
...      ...         ...
4868   CLA.L  2024-02-29
4869  UC87.L  2024-02-29
4870   EUA.L  2024-02-29
4871   ROR.L  2024-02-29
4872  0HOB.L  2024-02-29

[4873 rows x 2 columns]


0DRH.L: No price data found, symbol may be delisted (1d 2024-01-18 -> 2024-03-03)


No new data found for 0DRH.L since 2024-01-18. Skipping...


0OJX.L: No price data found, symbol may be delisted (1d 2024-02-14 -> 2024-03-03)


...

NMX.L: Period '1mo' is invalid, must be one of ['1d', '5d']


No new data found for NMX.L since 2014-08-02. Skipping...


38OG.L: Period '1mo' is invalid, must be one of ['1d', '5d']


No new data found for 38OG.L since 2019-01-23. Skipping...


IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
from datetime import datetime
current_date = datetime.now().date()
print("ETL update Complete", current_date)

ETL update Complete 2024-03-03
